# Inline Interactive Graph

In [7]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiLine
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html
import IPython

# needed if using show() in some browser installations
# (not this exact location, of course; but something the
#  browser can access)
# output_file("/home/peter/Documents/cs183/tmp/sinslider.html")

ECHARGE = 1
CG1 = 1
CG2 = 1
CL = 1
CR = 1

CM = 1

C1 = CG1 + CL + CM
C2 = CG2 + CR + CM
EC1 = ECHARGE * ECHARGE / C1 / (1 - CM * CM / C1 / C2)
EC2 = ECHARGE * ECHARGE / C2 / (1 - CM * CM / C2 / C1)
ECM = ECHARGE * ECHARGE / CM / (C1 * C2 / CM / CM - 1)

XLIM = 4
YLIM = 4

v1t1 = []
v2t1 = []
v1t2 = []
v2t2 = []
hexx = []
hexy = []
x1start = ECHARGE * (EC2 / 2 + 0 * ECM - EC1 * EC2 / ECM * (0 + .5)) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM)
y1start = ECHARGE * (EC1 / 2 + 0 * ECM - EC2 * EC1 / ECM * (0 + .5)) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM)
x2start = ECHARGE * (-EC2 / 2 + 1 * ECM - EC1 * EC2 / ECM * (1 - .5)) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM)
y2start = ECHARGE * (-EC1 / 2 + 1 * ECM - EC2 * EC1 / ECM * (1 - .5)) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM)
deltax = (ECM * ECM - EC1 * EC2) / (ECM * CG1 * ECM - CG1 * EC1 * EC2)
deltay = (ECM * ECM - EC1 * EC2) / (ECM * CG2 * ECM - CG2 * EC1 * EC2)
y1start_save = y1start
y2start_save = y2start
while x1start < XLIM + deltax:
    while y1start < YLIM + deltay:
        v1t1.append(x1start)
        v2t1.append(y1start)
        v1t2.append(x2start)
        v2t2.append(y2start)
        hexx.append( [x1start, x2start, x1start, x2start-deltax, x1start-deltax, x2start-deltax, x1start] )
        hexy.append( [y1start, y2start-deltay, y1start-deltay, y2start-deltay, y1start, y2start, y1start] )
        y1start += deltay
        y2start += deltay
    x1start += deltax
    x2start += deltax
    y1start = y1start_save
    y2start = y2start_save

source = ColumnDataSource(data=dict(v1t1=v1t1, v2t1=v2t1, v1t2=v1t2, v2t2=v2t2, hexx=hexx, hexy=hexy))

plot = figure(y_range=(0, YLIM), x_range=(0, XLIM), width=400, height=400,
              x_axis_label=r"$$V_1\text{ (arbitrary units)}$$",
              y_axis_label=r"$$V_2\text{ (arbitrary units)}$$")

hexagons = MultiLine(xs='hexx', ys='hexy', line_color="black")
plot.add_glyph(source, hexagons)
plot.circle(x='v1t1', y='v2t1', color="blue", source=source)
plot.circle(x='v1t2', y='v2t2', color="red", source=source)

cm_slider = Slider(start=0.02, end=10, value=1, step=.02, title="C_M")
cg1_slider = Slider(start=0.4, end=2, value=1, step=.02, title="C_g1")
cg2_slider = Slider(start=0.4, end=2, value=1, step=.02, title="C_g2")
cl_slider = Slider(start=0.02, end=10, value=1, step=.02, title="C_L")
cr_slider = Slider(start=0.02, end=10, value=1, step=.02, title="C_R")

callback = CustomJS(args=dict(source=source, ECHARGE=ECHARGE, cg1=cg1_slider, cg2=cg2_slider,
                              cl=cl_slider, cr=cr_slider, cm=cm_slider, XLIM=XLIM, YLIM=YLIM),
                    code="""
    const CM = cm.value
    const CG1 = cg1.value
    const CG2 = cg2.value
    const CR = cr.value
    const CL = cl.value

    const C1 = CG1 + CL + CM
    const C2 = CG2 + CR + CM
    const EC1 = ECHARGE * ECHARGE / C1 / (1 - CM * CM / C1 / C2)
    const EC2 = ECHARGE * ECHARGE / C2 / (1 - CM * CM / C2 / C1)
    const ECM = ECHARGE * ECHARGE / CM / (C1 * C2 / CM / CM - 1)

    const v1t1 = []
    const v2t1 = []
    const v1t2 = []
    const v2t2 = []
    const hexx = []
    const hexy = []
    var x1start = ECHARGE * (EC2 / 2 + 0 * ECM - EC1 * EC2 / ECM * (0 + .5)) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM)
    var y1start = ECHARGE * (EC1 / 2 + 0 * ECM - EC2 * EC1 / ECM * (0 + .5)) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM)
    var x2start = ECHARGE * (-EC2 / 2 + 1 * ECM - EC1 * EC2 / ECM * (1 - .5)) / (CG1 * ECM - CG1 * EC1 * EC2 / ECM)
    var y2start = ECHARGE * (-EC1 / 2 + 1 * ECM - EC2 * EC1 / ECM * (1 - .5)) / (CG2 * ECM - CG2 * EC2 * EC1 / ECM)
    const deltax = (ECM * ECM - EC1 * EC2) / (ECM * CG1 * ECM - CG1 * EC1 * EC2)
    const deltay = (ECM * ECM - EC1 * EC2) / (ECM * CG2 * ECM - CG2 * EC1 * EC2)
    const y1start_save = y1start
    const y2start_save = y2start
    while (x1start < XLIM + deltax) {
        while (y1start < YLIM + deltay) {
            v1t1.push(x1start)
            v2t1.push(y1start)
            v1t2.push(x2start)
            v2t2.push(y2start)
            hexx.push( [x1start, x2start, x1start, x2start-deltax, x1start-deltax, x2start-deltax, x1start] )
            hexy.push( [y1start, y2start-deltay, y1start-deltay, y2start-deltay, y1start, y2start, y1start] )
            y1start += deltay
            y2start += deltay
        }
        x1start += deltax
        x2start += deltax
        y1start = y1start_save
        y2start = y2start_save
    }

    source.data = { v1t1, v2t1, v1t2, v2t2, hexx, hexy }
""")

cm_slider.js_on_change('value', callback)
cg1_slider.js_on_change('value', callback)
cg2_slider.js_on_change('value', callback)
cl_slider.js_on_change('value', callback)
cr_slider.js_on_change('value', callback)

html_repr = file_html(row(plot, column(cm_slider, cg1_slider, cg2_slider, cl_slider, cr_slider)), CDN)
IPython.display.HTML(html_repr)
# debugging
# show(row(plot, column(cm_slider, cg1_slider, cg2_slider, cl_slider, cr_slider)))